# 0.0. IMPORTS

In [1]:
import pandas as pd
import numpy as np
import inflection
import math
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import HTML
from IPython.display import Image
import datetime

## 0.1. Helper Functions

## 0.2. Loading data

In [2]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

In [3]:
#merge:
df_raw = pd.merge (df_sales_raw, df_store_raw, how='left', on= 'Store')
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
769762,83,6,2013-08-10,2667,260,1,0,0,0,a,a,2710.0,NaN,NaN,0,NaN,NaN,NaN


# 1.0. DESCRIÇÃO DE DADOS

## 1.1. Rename Columns

In [4]:
df1 = df_raw.copy()

In [5]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']
snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

#rename
df1.columns = cols_new

## 1.2. Data Dimension

In [6]:
print("Number of Rows: {}".format(df1.shape[0]))
print("Number of Collumns: {}".format(df1.shape[1]))

Number of Rows: 1017209
Number of Collumns: 18


## 1.3. Data Types

In [7]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [8]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4. Check NA

In [9]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Fillout NA

In [10]:
#competition_distance
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)

#competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

#competition_open_since_year     
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

#promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)
              
#promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)



In [11]:
#promo_interval
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'May', 5: 'Apr', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
df1['promo_interval'].fillna(0, inplace=True)
df1['month_map'] = df1['date'].dt.month.map(month_map)
df1['is_promo'] = df1.apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in str(x['promo_interval']).split(',') else 0, axis =1)

In [12]:
df1.sample(5).T

,650957,210959,655282,736696,492992
store,583,225,448,467,948
day_of_week,1,5,4,1,3
date,2013-11-25 00:00:00,2015-01-23 00:00:00,2013-11-21 00:00:00,2013-09-09 00:00:00,2014-04-16 00:00:00
sales,3762,3849,7721,18636,6575
customers,447,437,727,1967,1703
open,1,1,1,1,1
promo,0,0,1,1,1
state_holiday,0,0,0,0,0
school_holiday,0,0,0,0,1
store_type,a,d,a,a,b


## 1.6 Change types

In [13]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [14]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype('int64')
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype('int64')
df1['promo2_since_week'] = df1['promo2_since_week'].astype('int64')
df1['promo2_since_year'] = df1['promo2_since_year'].astype('int64')

In [15]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month             int64
competition_open_since_year              int64
promo2                                   int64
promo2_since_week                        int64
promo2_since_year                        int64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

## 1.7. Descriptive Statistical 

In [16]:
num_attributes = df1.select_dtypes (include=['int64', 'float64'])
cat_attributes = df1.select_dtypes (exclude= ['int64', 'float64', 'datetime64[ns]'])

## 1.7.1 Numerical Attributes

In [ ]:
#Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#Dispersion - std, min, max,range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() ) ).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
                  
#concatenate
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns=['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

In [ ]:
sns.displot(df1['sales'])

## 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
aux1 = df1[(df1['state_holiday'] != 0) & (df1['sales'] > 0)]

plt.subplot(1, 3, 1)
sns.boxplot(x='state_holiday', y='sales', data=aux1)

plt.subplot(1, 3, 2)
sns.boxplot(x='store_type', y='sales', data=aux1)

plt.subplot(1, 3, 3)
sns.boxplot(x='assortment', y='sales', data=aux1)

# 2.0. FEATURE ENGINEERING

## 2.1. Mapa mental de Hipóteses

In [ ]:
Image('img/Daily_Store_Sales.png')

## 2.2 Criação de Hipóteses

In [ ]:
df2 = df1.copy()

### 2.2.1. Hipóteses da Loja

<ol>
    <li>Lojas com número maior de funcionários deveriam vender mais; </li>
    <li>Lojas com maior capacidade de estoque deveriam vender mais; </li>
    <li>Lojas de maior porte deveriam vender mais; </li>
    <li>Lojas com maior diversidade de produtos deveriam vender mais; </li>
    <li>Lojas com competidores mais próximos deveriam vender menos; </li>
    <li>Lojas com competidores de longa data deveriam vender mais; </li>
</ol>


### 2.2.2. Hipóteses do Produto

<ol>
    <li>Lojas que investem mais em marketng deveriam  vender mais;</li>
    <li>Lojas com maior exposição de produtos deveriam vender mais; </li>
    <li>Lojas com produtos de preços menores deveriam vender mais; </li>
    <li>Lojas com promoções mais agressivas (descontos maiores) deveriam vender mais; </li>
    <li>Lojas com mais dias de promoção deveriam vender mais; </li>
    <li>Lojas com mais promoções consecutivas deveriam vender mais </li>
</ol>

### 2.2.3. Hipóteses Temporais

<ol>
    <li>Lojas abertas durante o feriado de natal deveriam vender mais</li>
    <li>Lojas deveriam vender mais ao longo dos anos; </li>
    <li>Lojas deveriam vender mais no segundo semestre do ano </li>
    <li>Lojas deveriam vender mais depois do dia 10 de cada mês </li>
    <li>Lojas deveriam vender menos aos finais de semana </li>
    <li>Lojas deveriam vender menos durante os feriados escolares </li>
</ol>

## 2.3. Lista Final de Hipóteses

<ol>
    <li>Lojas com maior diversidade de produtos deveriam vender mais; </li>
    <li>Lojas com competidores mais próximos deveriam vender menos; </li>
    <li>Lojas com competidores de longa data deveriam vender mais; </li>
    <li>Lojas com produtos de preços menores deveriam vender mais; </li>
    <li>Lojas com promoções mais agressivas (descontos maiores) deveriam vender mais; </li>
    <li>Lojas com mais dias de promoção deveriam vender mais; </li>
    <li>Lojas com mais promoções consecutivas deveriam vender mais </li>
    <li>Lojas abertas durante o feriado de natal deveriam vender mais</li>
    <li>Lojas deveriam vender mais ao longo dos anos; </li>
    <li>Lojas deveriam vender mais no segundo semestre do ano </li>
    <li>Lojas deveriam vender mais depois do dia 10 de cada mês </li>
    <li>Lojas deveriam vender menos aos finais de semana </li>
    <li>Lojas deveriam vender menos durante os feriados escolares </li>
</ol>

## 2.3. Feature Engineering

In [ ]:
#year
df2['year']= df2['date'].dt.year
#month
df2['month']= df2['date'].dt.month
#day
df2['day']= df2['date'].dt.day
#week of year
df2['week_of_year']= df2['date'].dt.isocalendar().week
#year week
df2['year_week']= df2['date'].dt.strftime('%Y-%W')

#competition since
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

#promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-'+ df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

#assortment
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x=='a' else 'extra' if x=='b' else 'extended')

#state holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x=='a' else 'easter_holiday' if x=='b' else 'christmas' if x=='c' else 'regular_day')

In [ ]:
df2.head().T

# 3.0. FILTRAING FEATURES

In [ ]:
df3 = df2.copy()

### 3.1. Filtragem de linhas

Nessa setor eu vou colocar todas as variáveis que restrigem o negócio:
A exemplo: a variável 'open' será só 0 ou 1, 0 no caso de a loja estar fechada.
A segunda variável é a de 'sales' em que 0 significa: não houve venda.

In [ ]:
df3 =df3[(df3['open'] !=0) & df3['sales']>0]

### 3.2. Seleção de Colunas

Nessa seleção de colunas eu posso retirar as que não vão mais gerar novas ideias ou insights de negócios. Sendo pq as ideias principais já foram retiradas dos dados contidos nela ou por que não são mais necessários

In [ ]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis =1)

In [ ]:
df3.head()

# 4.0. EXPLORATORY DATA ANALYSIS

In [ ]:
df4=df3.copy()

## 4.1. Análise Univariada

### 4.1.1. Response Variable

In [ ]:
sns.histplot(df4['sales'], size=20);

### 4.1.2. Numerical Variable

In [ ]:
num_attributes.hist();

### 4.1.3. Categorical Variable

In [ ]:
cat_attributes.head()

In [ ]:
# state_holiday
# store_type
# assortment